In [53]:
from google.cloud import bigquery
import os

path = "/content/credentials.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path

client = bigquery.Client()
dataset_ref = client.dataset("iowa_liquor_sales", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("sales")
table = client.get_table(table_ref)

df = client.list_rows(table=table, max_results=1000000).to_dataframe()
print(df.head(20))
print(df.describe())
print(df.columns.tolist())



   invoice_and_item_number        date  ... volume_sold_liters volume_sold_gallons
0             S26345100049  2015-06-22  ...               2.40                0.63
1             S17774100094  2014-03-10  ...             192.00               50.72
2             S08249300002  2012-10-11  ...              22.50                5.94
3             S06239100038  2012-06-25  ...             262.50               69.35
4          INV-20674100062  2019-07-17  ...              27.00                7.13
5          INV-20928700084  2019-07-30  ...               2.00                0.52
6             S17599100038  2014-02-26  ...             108.00               28.53
7          INV-25434400003  2020-02-25  ...             126.00               33.28
8          INV-25495300001  2020-02-27  ...              22.50                5.94
9          INV-25493400001  2020-02-27  ...               6.00                1.58
10         INV-20097500048  2019-06-19  ...               2.00                0.52
11  

# Data Cleaning

In [100]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from plotly.offline import init_notebook_mode, iplot
%matplotlib inline

In [55]:
df.head(50)


,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S26345100049,2015-06-22,2528,Hy-Vee Food Store #3 / Des Moines,3221 SE 14TH ST,DES MOINES,50315,None,77,Polk,1081380,MISCELLANEOUS SCHNAPPS,55,Sazerac North America,84172,99 Bananas Mini,10,600,5.94,8.91,4,35.64,2.40,0.63
1,S17774100094,2014-03-10,3385,Sam's Club 8162 / Cedar Rapids,2605 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.67969 42.031819),57,Linn,1081400,TRIPLE SEC,55,Sazerac North America,86390,Montezuma Triple Sec,12,1000,2.13,3.19,192,612.48,192.00,50.72
2,S08249300002,2012-10-11,4719,Walgreens #06677 / West Des Moines,4900 MILLS CIVIC PKWY,WEST DES MOINES,50265,POINT (-93.770018 41.559128),77,Polk,1011500,STRAIGHT RYE WHISKIES,255,Wilson Daniels Ltd.,27102,Templeton Rye,6,750,18.08,27.13,30,813.90,22.50,5.94
3,S06239100038,2012-06-25,2638,Hy-Vee Wine and Spirits / Cedar Fall,5925 UNIVERSITY AVE STE A,CEDAR FALLS,50613,POINT (-92.429331 42.512766),07,Black Hawk,1701100,DECANTERS & SPECIALTY PACKAGES,260,Diageo Americas,3402,Captain Morgan Original Spiced Barrel Tray Pack,6,1750,17.50,26.25,150,3937.50,262.50,69.35
4,INV-20674100062,2019-07-17,2512,Hy-Vee Wine and Spirits / Iowa City,1720 Waterfront Dr,Iowa City,52240,POINT (-91.53046300000001 41.642764),52,JOHNSON,1091200,Neutral Grain Spirits,434,LUXCO INC,41846,Everclear Alcohol,12,750,9.00,13.50,36,486.00,27.00,7.13
5,INV-20928700084,2019-07-30,2651,Hy-Vee / Waverly,1311 4 Street SW,Waverly,50677,POINT (-92.475403 42.712263),9,BREMER,1062100,Gold Rum,434,LUXCO INC,45247,Paramount Gold Rum,12,1000,4.42,6.63,2,13.26,2.00,0.52
6,S17599100038,2014-02-26,3524,Sam's Club 6568 / Ames,305 AIRPORT RD,AMES,50010,POINT (-93.613648 42.001123),85,Story,1081700,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.76,7.14,108,771.12,108.00,28.53
7,INV-25434400003,2020-02-25,3814,Costco Wholesale #788 / WDM,7205 Mills Civic Pkwy,West Des Moines,50266,POINT (-93.805651 41.562369),25,Dallas,1012300,Single Malt Scotch,266,Edrington Group USA LLC,5555,Macallan Double Cask 12YR,12,750,34.99,52.49,168,8818.32,126.00,33.28
8,INV-25495300001,2020-02-27,2561,Hy-Vee Food Store / Fleur / DSM,4605 Fleur Drive,Des Moines,50321,POINT (-93.64494300000001 41.542748),77,POLK,1011600,Straight Rye Whiskies,115,CONSTELLATION BRANDS INC,27094,High West Rendezvous Rye,6,750,37.50,56.25,30,1687.50,22.50,5.94
9,INV-25493400001,2020-02-27,2675,Hy-Vee #2 / Coralville,3285 Crosspark Rd,Coralville,52241,POINT (-91.60627100000002 41.721438),52,JOHNSON,1901200,Special Order Items,065,Jim Beam Brands,919471,Makers Mark Mini,10,600,16.44,24.66,10,246.60,6.00,1.58


In [56]:
df.dtypes

invoice_and_item_number     object
date                        object
store_number                object
store_name                  object
address                     object
city                        object
zip_code                    object
store_location              object
county_number               object
county                      object
category                    object
category_name               object
vendor_number               object
vendor_name                 object
item_number                 object
item_description            object
pack                         int64
bottle_volume_ml             int64
state_bottle_cost          float64
state_bottle_retail        float64
bottles_sold                 int64
sale_dollars               float64
volume_sold_liters         float64
volume_sold_gallons        float64
dtype: object

In [57]:
df['city'] = df['city'].str.lower()
print(df['city'].nunique())
df['city'].unique()


454


array(['des moines', 'cedar rapids', 'west des moines', 'cedar falls',
       'iowa city', 'waverly', 'ames', 'coralville', 'keokuk',
       'sioux city', 'ankeny', 'spencer', 'onawa', 'davenport',
       'humboldt', 'hampton', 'storm lake', 'bettendorf', 'fairfield',
       'waterloo', 'johnston', 'dubuque', 'decorah', 'council bluffs',
       'leclaire', 'marion', 'urbandale', 'belle plaine', 'waukee',
       'rock valley', 'waukon', 'osceola', 'ottuwma', 'grimes', 'cresco',
       'melcher-dallas', 'washington', 'red oak', 'winterset', 'atlantic',
       'newton', 'mason city', 'clear lake', 'jefferson', 'coon rapids',
       'marshalltown', 'bancroft', 'monticello', 'blue grass',
       'windsor heights', 'pella', 'marengo', 'creston', 'indianola',
       'mt vernon', 'manchester', 'clinton', 'emmetsburg',
       'missouri valley', 'tipton', 'independence', 'altoona',
       'arnolds park', 'pleasant hill', None, 'spirit lake',
       'mount vernon', 'ida grove', 'north liberty', '

In [58]:
df['city'].value_counts()

des moines      84999
cedar rapids    65042
davenport       44820
waterloo        31354
iowa city       29362
                ...  
earling             8
wyoming             7
nichols             7
sageville           7
rudd                6
Name: city, Length: 454, dtype: int64

In [59]:
df.isnull().sum()

invoice_and_item_number        0
date                           0
store_number                   0
store_name                     0
address                     4050
city                        4050
zip_code                    4052
store_location             95536
county_number               7923
county                      7923
category                    1418
category_name               2065
vendor_number                  0
vendor_name                    0
item_number                    0
item_description               0
pack                           0
bottle_volume_ml               0
state_bottle_cost              0
state_bottle_retail            0
bottles_sold                   0
sale_dollars                   0
volume_sold_liters             0
volume_sold_gallons            0
dtype: int64

#### Let's delete column 'store location'. This column is useless for our analysis and also contain a lot of missing values. 

In [60]:
df = df.drop('store_location', 1).copy()
df.isnull().sum()

invoice_and_item_number       0
date                          0
store_number                  0
store_name                    0
address                    4050
city                       4050
zip_code                   4052
county_number              7923
county                     7923
category                   1418
category_name              2065
vendor_number                 0
vendor_name                   0
item_number                   0
item_description              0
pack                          0
bottle_volume_ml              0
state_bottle_cost             0
state_bottle_retail           0
bottles_sold                  0
sale_dollars                  0
volume_sold_liters            0
volume_sold_gallons           0
dtype: int64

#### There are also missed values in other columns. Their amount relatively to size of the dataset is quite low thats why we can just delete rows which contain missing values.

In [61]:
df.dropna(inplace = True)
df.shape

(990042, 23)

In [62]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [63]:
df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,year,month,day
0,S26345100049,2015-06-22,2528,Hy-Vee Food Store #3 / Des Moines,3221 SE 14TH ST,des moines,50315,77,Polk,1081380,MISCELLANEOUS SCHNAPPS,55,Sazerac North America,84172,99 Bananas Mini,10,600,5.94,8.91,4,35.64,2.4,0.63,2015,6,22
1,S17774100094,2014-03-10,3385,Sam's Club 8162 / Cedar Rapids,2605 BLAIRS FERRY RD NE,cedar rapids,52402,57,Linn,1081400,TRIPLE SEC,55,Sazerac North America,86390,Montezuma Triple Sec,12,1000,2.13,3.19,192,612.48,192.0,50.72,2014,3,10
2,S08249300002,2012-10-11,4719,Walgreens #06677 / West Des Moines,4900 MILLS CIVIC PKWY,west des moines,50265,77,Polk,1011500,STRAIGHT RYE WHISKIES,255,Wilson Daniels Ltd.,27102,Templeton Rye,6,750,18.08,27.13,30,813.90,22.5,5.94,2012,10,11
3,S06239100038,2012-06-25,2638,Hy-Vee Wine and Spirits / Cedar Fall,5925 UNIVERSITY AVE STE A,cedar falls,50613,07,Black Hawk,1701100,DECANTERS & SPECIALTY PACKAGES,260,Diageo Americas,3402,Captain Morgan Original Spiced Barrel Tray Pack,6,1750,17.50,26.25,150,3937.50,262.5,69.35,2012,6,25
4,INV-20674100062,2019-07-17,2512,Hy-Vee Wine and Spirits / Iowa City,1720 Waterfront Dr,iowa city,52240,52,JOHNSON,1091200,Neutral Grain Spirits,434,LUXCO INC,41846,Everclear Alcohol,12,750,9.00,13.50,36,486.00,27.0,7.13,2019,7,17


In [65]:
print(df['date'].min())
print(df['date'].max())

2012-01-03 00:00:00
2020-09-30 00:00:00


In [66]:
print(df['category_name'].nunique())
df['category_name'].unique()

103


array(['MISCELLANEOUS SCHNAPPS', 'TRIPLE SEC', 'STRAIGHT RYE WHISKIES',
       'DECANTERS & SPECIALTY PACKAGES', 'Neutral Grain Spirits',
       'Gold Rum', 'DISTILLED SPIRITS SPECIALTY', 'Single Malt Scotch',
       'Straight Rye Whiskies', 'Special Order Items', 'Aged Dark Rum',
       'CHERRY BRANDIES', 'Triple Sec', 'WATERMELON SCHNAPPS',
       'COFFEE LIQUEURS', 'Corn Whiskies', 'IRISH WHISKIES',
       'PEACH SCHNAPPS', 'Coffee Liqueurs', 'ROOT BEER SCHNAPPS',
       'SINGLE MALT SCOTCH', 'CINNAMON SCHNAPPS',
       'TROPICAL FRUIT SCHNAPPS', 'APPLE SCHNAPPS',
       'Neutral Grain Spirits Flavored',
       'American Distilled Spirit Specialty', 'APRICOT BRANDIES',
       'AMERICAN AMARETTO', 'Single Barrel Bourbon Whiskies',
       'Imported Distilled Spirit Specialty', 'American Sloe Gins',
       'IMPORTED AMARETTO', 'GRAPE SCHNAPPS', 'BOTTLED IN BOND BOURBON',
       'PEACH BRANDIES', 'BARBADOS RUM', 'BUTTERSCOTCH SCHNAPPS',
       'American Cordials & Liqueurs', 'BLACKBERRY

In [67]:
df['category_name'] = df['category_name'].str.lower()
df['category_name'].unique()
df['category_name'].value_counts()


american vodkas                         120818
vodka 80 proof                          104564
spiced rum                               84164
blended whiskies                         70376
whiskey liqueur                          65801
                                         ...  
imported distilled spirits specialty         7
temporary  & specialty packages              5
high proof beer - american                   5
holiday vap                                  3
imported vodka - cherry                      1
Name: category_name, Length: 86, dtype: int64

In [68]:
print(df['item_description'].nunique())
df['item_description'].value_counts()

4022


Hawkeye Vodka                             37104
Five O'clock Vodka                        21032
Fireball Cinnamon Whiskey                 19308
Captain Morgan Spiced Rum                 17516
Admiral Nelson Spiced Rum                 16534
                                          ...  
O'Dowd's Pumpkin Spice                        1
Ypioca Cachaca Crystal                        1
Russell's Reserve Single Barrel Rye HA        1
Glenkinchie 12yr Single Malt Scotch           1
SOOH 1792 Bottled in Bond Bourbon             1
Name: item_description, Length: 4022, dtype: int64

### Basic analysis

top 10 sales$ depending on category

In [72]:
df_plot = df.groupby(['category_name'])['sale_dollars'].sum().sort_values(ascending=False)
top_count = pd.DataFrame(df_plot)
top_count1 = pd.DataFrame(df_plot.head(10))

In [73]:


fig_reg = px.bar(top_count1,x=top_count1.index, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Sales of liquor per category",
    xaxis_title=" Category Name",
    yaxis_title="Sales in dollars",
    )
fig_reg.show()

top10 sales dependent on city

In [80]:
df_plot = df.groupby(['city'])['sale_dollars'].sum().sort_values(ascending=False)
top_count = pd.DataFrame(df_plot)
top_count1 = pd.DataFrame(df_plot.head(10))


city
des moines         1.550309e+07
cedar rapids       8.692251e+06
davenport          6.510629e+06
west des moines    5.158377e+06
iowa city          4.462717e+06
Name: sale_dollars, dtype: float64

In [77]:
fig_reg = px.bar(top_count1,x=top_count1.index, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Sales of liquor per city",
    xaxis_title=" city",
    yaxis_title="Sales in dollars",
    )
fig_reg.show()

In [94]:
def transform(a):
  return datetime.date(1900, a, 1).strftime('%B')

dependence on day and sales

In [95]:
df_plot = df.groupby(['month'])['sale_dollars'].sum().sort_values(ascending=False)
top_count = pd.DataFrame(df_plot)
new = []
for i in top_count.index:
  new.append(transform(i))

['December', 'June', 'May', 'October', 'August', 'September', 'July', 'April', 'March', 'November', 'February', 'January']


In [96]:
fig_reg = px.bar(top_count,x=new, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Sales of liquor per month",
    xaxis_title=" Month",
    yaxis_title="Sales in dollars",
    )
fig_reg.show()

In [98]:
daily_sales = df.groupby('date', as_index=False)['sale_dollars'].sum()

In [101]:
daily_sales_sc = go.Scatter(x=daily_sales['date'], y=daily_sales['sale_dollars'])
layout = go.Layout(title='Daily sales', xaxis=dict(title='date'), yaxis=dict(title='sales'))
fig = go.Figure(data=[daily_sales_sc], layout=layout)
iplot(fig)